In [1]:

!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 24.5 MB/s eta 0:00:00
   ━

We will also download **Soundfile and librosa** to be able to work with sound files in Python. The librosa library may need to have ffmpeg installed. This page on librosa provides installation instructions for FFmpeg.

Finally, we will download the **Gardio** package that we will use by the end of this article to demo and deploy the application we will build throughout this application.

In [2]:
from datasets import load_dataset
dataset = load_dataset("librispeech_asr",
                       split="train.clean.100",  # refers to a training subset of LibriSpeech with clean audio and approximately 100 hours of recordings.
                       streaming=True,  # Streaming can provide records as Python objects, one at a time, without loading the entire dataset into memory
                       trust_remote_code=True)  # This flag allows the execution of custom remote code defined by the dataset creator.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

librispeech_asr.py:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

**LibriSpeech** is a corpus of approximately 1000 hours of 16kHz read English speech, prepared by Vassil Panayotov with the assistance of Daniel Povey. The data is derived from reading audiobooks from the LibriVox project and has been carefully segmented and aligned.

In [3]:
example = next(iter(dataset))  #
dataset_head = dataset.take(3)
list(dataset_head)

[{'file': '374-180298-0000.flac',
  'audio': {'path': '374-180298-0000.flac',
   'array': array([ 7.01904297e-04,  7.32421875e-04,  7.32421875e-04, ...,
          -2.74658203e-04, -1.83105469e-04, -3.05175781e-05]),
   'sampling_rate': 16000},
  'text': 'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED',
  'speaker_id': 374,
  'chapter_id': 180298,
  'id': '374-180298-0000'},
 {'file': '374-180298-0001.flac',
  'audio': {'path': '374-180298-0001.flac',
   'array': array([-9.15527344e-05, -1.52587891e-04, -1.52587891e-04, ...,
          -2.13623047e-04, -1.83105469e-04, -2.74658203e-04]),
   'sampling_rate': 16000},
  'text': "MARGUERITE TO BE UNABLE TO LIVE APART FROM ME IT WAS THE DAY AFTER THE EVENING WHEN SHE CAME TO SEE ME THAT I SENT HER MANON LESCAUT FROM THAT TIME SEEING THAT I COULD NOT CHANGE MY MISTRESS'S LIFE I CHANGED MY OWN",
  'speaker_id': 374

In [8]:
list(dataset_head)[1]

{'file': '374-180298-0001.flac',
 'audio': {'path': '374-180298-0001.flac',
  'array': array([-9.15527344e-05, -1.52587891e-04, -1.52587891e-04, ...,
         -2.13623047e-04, -1.83105469e-04, -2.74658203e-04]),
  'sampling_rate': 16000},
 'text': "MARGUERITE TO BE UNABLE TO LIVE APART FROM ME IT WAS THE DAY AFTER THE EVENING WHEN SHE CAME TO SEE ME THAT I SENT HER MANON LESCAUT FROM THAT TIME SEEING THAT I COULD NOT CHANGE MY MISTRESS'S LIFE I CHANGED MY OWN",
 'speaker_id': 374,
 'chapter_id': 180298,
 'id': '374-180298-0001'}

In [12]:
from IPython.display import Audio
Audio(data=list(dataset_head)[1]["audio"]["array"],
      autoplay=True,
      rate=list(dataset_head)[1]["audio"]["sampling_rate"])

# Build Transformers Pipeline

In [13]:
from transformers import pipeline
pipe = pipeline("automatic-speech-recognition", model="distil-whisper/distil-small.en")

config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/332M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Device set to use cpu


In [15]:
pipe.feature_extractor.sampling_rate

16000

The sampling rate expected by the speech recognition model is 16000. We have to make sure that the data sampling rate is the same, or else we will need to apply transformation for it.

In [17]:
example["audio"]["sampling_rate"]

16000

Using the model to recognize the audio and get the text

In [18]:
pipe(list(dataset_head)[1]["audio"]["array"])

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{'text': " Marguerite to be unable to live apart from me. It was the day after the evening when she came to see me that I sent her Manan Les Scott. From that time, seeing that I could not change my mistress's life, I changed my own."}

# Deploy Application Demo with Gradio

In [19]:
import os
import gradio as gr
demo = gr.Blocks()

**Gradio Overview**
Gradio is a Python library used for creating interactive user interfaces (UIs) for machine learning models, APIs, or any Python function. It provides an easy way to create web-based apps for demonstrations, testing, or sharing ML models.

**gr.Blocks**
gr.Blocks is a high-level API in Gradio that allows you to build complex UIs by combining multiple UI components (like text boxes, buttons, sliders, etc.) in a flexible layout. It provides more control over the layout and styling compared to simpler Gradio interfaces.

Purpose of **gr.Blocks()**:
It serves as a container for UI components.
It organizes components in rows, columns, or groups, allowing for a structured layout.
demo
demo is the variable assigned to the Blocks instance.
You will add components (e.g., text inputs, buttons, images, etc.) and define interactions within this demo object.


**Next**, we will define the transcribe_speech function. The transcribe_speech function validates the audio file input, processes it using the pipeline to transcribe speech, and returns the transcribed text. This function can be seamlessly integrated into a Gradio interface to provide an easy-to-use web application for speech-to-text transcription.

In [22]:
def transcribe_speech(path):
  if path in None:
    return ('wrong path, please retry')
  output = pipe(path)
  return output['text']


The following code sets up a Gradio interface that allows users to transcribe speech captured from their microphone. When the user records audio, it is saved as a file, and the path to this file is passed to the transcribe_speech function. The transcribed text is then displayed in a textbox labeled "Transcription".

This code creates a simple and intuitive interface for real-time speech transcription:

**Input**: A microphone recording.

**Output**: A text transcription.

Gradio makes it easy to handle audio input and display the processed text output in a clean UI.

In [23]:
mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="microphone",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never")


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Next, we will set up a Gradio interface **that allows users to transcribe speech captured from their microphone.** When the user records audio, it is saved as a file, and the path to this file is passed to the transcribe_speech function. The transcribed text is then displayed in a textbox labeled "Transcription".

In [24]:
file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="upload",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never",
)

Finally, we will create a Gradio-based web application that has two main functionalities:

Transcribing audio from a microphone.

Transcribing uploaded audio files.

It uses a tabbed interface to switch between these two functionalities. The application is launched with sharing enabled, and it listens on a specific port defined by an environment variable. This setup is ideal for providing an interactive and user-friendly interface for speech-to-text transcription tasks.

In [26]:

# Your Gradio app (example)
demo = gr.TabbedInterface(
    [mic_transcribe, file_transcribe],
    ["Transcribe Microphone", "Transcribe Audio File"],
)

# Launch without specifying the port and without share
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06d519986596d7042c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Colab Limitation: Colab cannot access local hardware like microphones.
Run Locally: To use the microphone, run your app on your local machine.